### Caching using KV pairs - single prompt

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The top guess word is argmax of logits below.

In [4]:
input_string = "This is a test string which can be"
inputs = tokenizer(input_string, return_tensors='pt')
print ('logits of inputs =', inputs)

with torch.no_grad():
  output = model(**inputs)

print ("output size = ", output.logits.shape, "inputs size = ", inputs.input_ids.shape)

## calculate the next token id from logits
logits = output.logits[0,-1:]
next_token_id = logits.argmax()
next_token = tokenizer.decode(next_token_id)

print (input_string, " --> ", next_token)

logits of inputs = {'input_ids': tensor([[1212,  318,  257, 1332, 4731,  543,  460,  307]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
output size =  torch.Size([1, 8, 50257]) inputs size =  torch.Size([1, 8])
This is a test string which can be  -->   used


The words with largest possiblities are listed below.

In [5]:
for i in range(len(output.logits[0])):
  print ("\n", i+1, 'logits iteration \n==================')
  top_5 = torch.topk(output.logits[0][i], 5)
  for ind, val in zip(top_5.indices, top_5.values):
    print (val.numpy(), tokenizer.decode(ind))


 1 logits iteration 
-31.889215  is
-32.30777 ,
-32.7851 .
-33.207935 

-33.20856 's

 2 logits iteration 
-99.705635  a
-99.90979  the
-100.58379  not
-100.92179  what
-101.117355  why

 3 logits iteration 
-102.71866  very
-102.94689  great
-103.48176  good
-104.08529  big
-104.23586  huge

 4 logits iteration 
-81.32381  of
-82.422516  case
-82.59881  that
-82.91623  to
-82.97061  for

 5 logits iteration 
-95.90601  that
-96.18236 ,
-96.3554 .
-96.60895  for
-96.938965  and

 6 logits iteration 
-104.71456  is
-105.37446  will
-105.755066  contains
-106.033554  can
-106.06146  you

 7 logits iteration 
-79.511444  be
-82.350685  only
-82.9923  contain
-83.725174  also
-83.76289 't

 8 logits iteration 
-120.76004  used
-123.49073  passed
-123.92379  found
-124.189766  either
-124.22276  read


Using caching with the model inputs.

In [6]:
inputs
inputs["input_ids"] = next_token_id.reshape(1,1)
inputs["attention_mask"] = torch.cat([inputs["attention_mask"], torch.tensor([[1]])], dim=1)
inputs, next_token_id

({'input_ids': tensor([[973]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 tensor(973))

In [46]:
input_string = "The field of machine learning"
inputs = tokenizer(input_string, return_tensors='pt')
num_chars = 50

def generate_next_token(inputs):
  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits[0,-1:]
    next_token_id = logits.argmax()

  inputs["input_ids"] = torch.cat([inputs["input_ids"], next_token_id.reshape(1,1)], dim=1)
  inputs["attention_mask"] = torch.cat([inputs["attention_mask"], torch.tensor([[1]])], dim=1)

  return outputs.logits.argmax(), inputs

def generate_next_token_with_cache(input_values):
  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits[0,-1:]
    next_token_id = logits.argmax()

  next_inputs = {
      "input_ids": next_token_id.reshape((1, 1)),
      "attention_mask": torch.cat(
          [input_values["attention_mask"], torch.tensor([[1]])],
          dim=1),
      "past_key_values": outputs.past_key_values,
  }

  return outputs.logits.argmax(), next_inputs

total_time = 0

for i in range(num_chars):
  start = time.time()
  next_token_id, inputs = generate_next_token(inputs)
  end = time.time()
  total_time += end-start

print ('Inference time (without kv cache) = ', total_time, "s\n ================================================== \n", tokenizer.decode(inputs["input_ids"][0]), "\n")

total_time = 0
inputs = tokenizer(input_string, return_tensors='pt')

output_ids = inputs["input_ids"]

for i in range(num_chars):
  start = time.time()
  next_token_id, inputs = generate_next_token_with_cache(inputs)
  end = time.time()
  output_ids = torch.cat([output_ids, next_token_id.reshape(1,1)], dim=1)
  total_time += end-start

print ('Inference time = ', total_time, "s\n ================================================== \n", tokenizer.decode(output_ids[0]), "\n")

Inference time (without kv cache) =  9.905962228775024 s
 The field of machine learning is still in its infancy, but it is becoming more and more important to understand how machine learning can be used to improve the quality of life for people.

Machine learning is a new field of research that has been gaining traction in recent years. 

Inference time =  2.785890817642212 s
 The field of machine learning
 still in its infancy, but it is becoming more and more important to understand how machine learning can be used to improve the quality of life for people.

Machine learning is a new field of research that has been gaining traction in recent years. 



### Cacheing with kv pairs - multi prompt.

In [64]:
prompts = [
    "The quick brown fox jumped over the",
    "The rain in Spain falls",
    "What comes up must",
]

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

inputs = tokenizer(prompts, padding=True, return_tensors="pt")

attention_mask = inputs["attention_mask"]
position_ids = attention_mask.long().cumsum(-1) - 1
position_ids.masked_fill_(attention_mask == 0, 1)

with torch.no_grad():
    outputs = model(position_ids=position_ids, **inputs)
logits = outputs.logits

last_logits = logits[:, -1, :]
next_token_ids = last_logits.argmax(dim=1)
next_tokens = tokenizer.batch_decode(next_token_ids)

for idx, nt in enumerate(next_tokens):
  print (prompts[idx], " -->",nt)

The quick brown fox jumped over the  -->  fence
The rain in Spain falls  -->  on
What comes up must  -->  be


In [67]:
def generate_next_token(inputs):
  attn_mask = inputs["attention_mask"]
  position_ids = attn_mask.long().cumsum(-1) - 1
  position_ids.masked_fill_(attention_mask==0,1)

  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits[0,-1:]
    next_token_id = logits.argmax(dim=1)

  for idx,val  in enumerate(next_token_id):
    inputs["input_ids"] = torch.cat([inputs["input_ids"][idx], next_token_id.reshape(1,1)], dim=1)
    inputs["attention_mask"] = torch.cat([inputs["attention_mask"][idx], torch.tensor([[1]])], dim=1)

  next_tokens = tokenizer.batch_decode(next_token_ids)

  return outputs.logits.argmax(), inputs

generate_next_token(inputs)


(tensor(653628),
 {'input_ids': tensor([[  464,  2068,  7586, 21831, 11687,   625,   262],
         [50256, 50256,   464,  6290,   287,  8602,  8953],
         [50256, 50256, 50256,  2061,  2058,   510,  1276]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
         [0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 1, 1, 1, 1]])})